<a href="https://colab.research.google.com/github/alina775/20242R0136COSE47402/blob/main/FinalProject/deep_learning_final_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## PEFT initial example - https://www.datacamp.com/tutorial/llama3-fine-tuning-locally

In [ ]:
%%capture
%pip install -U transformers
%pip install -U datasets
%pip install -U accelerate
%pip install -U peft
%pip install -U trl
%pip install -U bitsandbytes
%pip install -U wandb

In [ ]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    DefaultDataCollator,
    Trainer,
    pipeline,
    logging,
)
from peft import (
    LoraConfig,
    PeftModel,
    prepare_model_for_kbit_training,
    get_peft_model,
)
import os, torch, wandb
from datasets import load_dataset
from trl import SFTTrainer, setup_chat_format

In [ ]:
from google.colab import userdata
wandb_key = userdata.get('wandb_key')
wandb.login(key=wandb_key)
run = wandb.init(
    project='Fine-tune Llama 3 8B on Medical Dataset',
    job_type="training",
    anonymous="allow"
)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: alina775 (deep_learning_final_project). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
from huggingface_hub import notebook_login
huggingface_key = userdata.get('huggingface')
notebook_login(huggingface_key)
#tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")
#model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")

base_model = "meta-llama/Llama-3.2-1B-Instruct"
dataset_name = "ruslanmv/ai-medical-chatbot"
new_model = "llama-3-8b-chat-doctor"

In [ ]:
torch_dtype = torch.float16
attn_implementation = "eager"

In [ ]:
# QLoRA config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch_dtype,
    bnb_4bit_use_double_quant=True,
)

# Load model
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map="auto",
    attn_implementation=attn_implementation
)

In [ ]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model)
tokenizer.chat_template = None
model, tokenizer = setup_chat_format(model, tokenizer)

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


In [ ]:
# LoRA config
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['up_proj', 'down_proj', 'gate_proj', 'k_proj', 'q_proj', 'v_proj', 'o_proj']
)
model = get_peft_model(model, peft_config)

In [ ]:
#Importing the dataset
dataset = load_dataset(dataset_name, split="all")
dataset = dataset.shuffle(seed=65).select(range(1000)) # Only use 1000 samples for quick demo

def format_chat_template(row):
    row_json = [{"role": "user", "content": row["Patient"]},
               {"role": "assistant", "content": row["Doctor"]}]
    row["text"] = tokenizer.apply_chat_template(row_json, tokenize=False)
    return row

dataset = dataset.map(
    format_chat_template,
    num_proc=4,
)

dataset['text'][3]

'<|im_start|>user\nFell on sidewalk face first about 8 hrs ago. Swollen, cut lip bruised and cut knee, and hurt pride initially. Now have muscle and shoulder pain, stiff jaw(think this is from the really swollen lip),pain in wrist, and headache. I assume this is all normal but are there specific things I should look for or will I just be in pain for a while given the hard fall?<|im_end|>\n<|im_start|>assistant\nHello and welcome to HCM,The injuries caused on various body parts have to be managed.The cut and swollen lip has to be managed by sterile dressing.The body pains, pain on injured site and jaw pain should be managed by pain killer and muscle relaxant.I suggest you to consult your primary healthcare provider for clinical assessment.In case there is evidence of infection in any of the injured sites, a course of antibiotics may have to be started to control the infection.Thanks and take careDr Shailja P Wahal<|im_end|>\n'

In [ ]:
dataset = dataset.train_test_split(test_size=0.1)

In [ ]:
training_arguments = TrainingArguments(
    output_dir=new_model,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=2,
    optim="paged_adamw_32bit",
    num_train_epochs=1,
    evaluation_strategy="steps",
    eval_steps=0.2,
    logging_steps=1,
    warmup_steps=10,
    logging_strategy="steps",
    learning_rate=2e-4,
    fp16=False,
    bf16=False,
    group_by_length=True,
    report_to="wandb"
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    peft_config=peft_config,
    max_seq_length=512,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_arguments,
    packing= False,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length, dataset_text_field. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:300: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:328: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will ove

Map:   0%|          | 0/900 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss,Validation Loss
90,3.474000,3.056747
180,2.721000,2.993811
270,2.480600,2.959504
360,3.015000,2.939756
450,2.849800,2.929710


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:257: UserWarning: Setting `save_embedding_layers` to `True` as the embed

TrainOutput(global_step=450, training_loss=2.9932316716512046, metrics={'train_runtime': 540.9588, 'train_samples_per_second': 1.664, 'train_steps_per_second': 0.832, 'total_flos': 1215122640052224.0, 'train_loss': 2.9932316716512046, 'epoch': 1.0})

In [ ]:
wandb.finish()
model.config.use_cache = True

eval/loss,█▅▃▂▁
eval/runtime,█▂▁▁▂
eval/samples_per_second,▁▇██▇
eval/steps_per_second,▁▇██▇
train/epoch,▁▁▁▂▂▂▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇▇█
train/global_step,▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇██
train/grad_norm,▃▄▂█▄▃▂▄▄▃▁▁▂▃▃▂▂▄▂▄▂▂▂▃▃▂▂▄▄▆▁▂▂▃▂▁▂▃▃▃
train/learning_rate,████▇▇▇▆▆▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▄▄▄▃▃▃▃▃▂▂▁▁▁▁
train/loss,█▆▆▃▆▆▅▆▇▆▅▆▅▇▄▅▅▄▅▄▃▄▆▄▇▄▁▄▅▄▅▅▄▄▃▅▄█▇▅
eval/loss,2.92971
eval/runtime,21.5564


## REFT inital example - https://medium.com/@syed_hasan/finetuning-llama-3-using-reft-representation-fine-tuning-technique-00f4fe1f497c

In [ ]:
try:
    import pyreft

except ModuleNotFoundError:
    !pip install git+https://github.com/stanfordnlp/pyreft.git

  Cloning https://github.com/stanfordnlp/pyreft.git to /tmp/pip-req-build-ntbf5ajj
  Running command git clone --filter=blob:none --quiet https://github.com/stanfordnlp/pyreft.git /tmp/pip-req-build-ntbf5ajj
  Resolved https://github.com/stanfordnlp/pyreft.git to commit b07868925d67e13efe6e222a6915e7ef0ce1e239
  Running command git submodule update --init --recursive -q
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.1 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of gcsfs to determine which version is compatible with other requirements. This could take a while.
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.3/293.3 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 55.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.9 MB/s eta 0:00:00
   ━━━

In [ ]:
%pip install transformers==4.45.2 sentence-transformers==3.1.1
#%pip install -U bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 12.8 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.47.0.dev0
    Uninstalling transformers-4.47.0.dev0:
      Successfully uninstalled transformers-4.47.0.dev0
  Attempting uninstall: sentence-transformers
    Found existing installation: sentence-transformers 3.2.1
    Uninstalling sentence-transformers-3.2.1:
      Successfully uninstalled sentence-transformers-3.2.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pyreft 0.0.8 requires transformers==4.45.1, but you have transformers 4.45.2 which is incompatible.


In [ ]:
import wandb
from google.colab import userdata
wandb_key = userdata.get('wandb_key')
wandb.login(key=wandb_key)
run = wandb.init(
    project='Fine-tune Llama 3 reft',
    job_type="training",
    anonymous="allow"
)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: alina775 (deep_learning_final_project). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
from huggingface_hub import notebook_login
huggingface_key = userdata.get('huggingface')
notebook_login(huggingface_key)

In [ ]:
import torch, transformers, pyreft
device = "cuda"

prompt_no_input_template = """<|begin_of_text|><|start_header_id|>user<|end_header_id|>%s<|eot_id|><|start_header_id|>assistant<|end_header_id|>"""

model_name_or_path = "meta-llama/Llama-3.2-1B-Instruct"
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_name_or_path, torch_dtype=torch.bfloat16, device_map=device, trust_remote_code=True)

# # get tokenizer
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_name_or_path, model_max_length=2048,
    padding_side="right", use_fast=False)
tokenizer.pad_token = tokenizer.eos_token

nnsight is not detected. Please install via 'pip install nnsight' for nnsight backend.


2024-11-29 09:17:11.992980: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-29 09:17:12.027724: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-29 09:17:12.038764: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-29 09:17:12.066262: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-29 09:17:14.381927: W tensorflow/comp

In [ ]:
# get reft model
reft_config = pyreft.ReftConfig(representations={
    "layer": 8, "component": "block_output",
    "low_rank_dimension": 4,
    "intervention": pyreft.LoreftIntervention(embed_dim=model.config.hidden_size,
    low_rank_dimension=4)})
reft_model = pyreft.get_reft_model(model, reft_config)
reft_model.set_device("cuda")
reft_model.print_trainable_parameters()

trainable intervention params: 16,388 || trainable model params: 0
model params: 1,235,814,400 || trainable%: 0.0013260890955794009


In [ ]:
dataset_name = "teknium/OpenHermes-2.5"
from datasets import load_dataset

dataset = load_dataset(dataset_name, split="all")
dataset = dataset.shuffle(seed=65).select(range(1_000))

data_module = pyreft.make_last_position_supervised_data_module(
    tokenizer, model, [prompt_no_input_template % row["conversations"][0]["value"] for row in dataset],
    [row["conversations"][1]["value"] for row in dataset])

In [ ]:
training_args = transformers.TrainingArguments(
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 2,
    warmup_steps = 10,
    num_train_epochs = 1,
    learning_rate = 5e-4,
    bf16 = True,
    logging_steps = 50,
    optim = "paged_adamw_32bit",
    weight_decay = 0.0,
    lr_scheduler_type = "cosine",
    output_dir = "outputs",
    report_to="wandb",
    fp16=False,
    logging_strategy="steps",
)


trainer = pyreft.ReftTrainerForCausalLM(model=reft_model, tokenizer=tokenizer, args=training_args, **data_module)

trainer.train()

Step,Training Loss
50,1.872600
100,1.970100
150,1.935300
200,1.838800
250,1.711800
300,1.766700
350,1.940800
400,2.238400
450,1.866000
500,1.845400


Directory 'outputs/checkpoint-500/intervenable_model' created successfully.


TrainOutput(global_step=500, training_loss=1.8985883178710938, metrics={'train_runtime': 611.1456, 'train_samples_per_second': 1.636, 'train_steps_per_second': 0.818, 'total_flos': 0.0, 'train_loss': 1.8985883178710938, 'epoch': 1.0})

In [ ]:
wandb.finish()
model.config.use_cache = True

train/epoch,▁▂▂▂▂▁▁▁▁▂▁▂▂▂▂▁▁▂▂▄▂▂▁▂▂▂▃▃▃▄▅▆▆▆▆▇██▁▂
train/global_step,▁▁▁▁▂▂▁▂▂▂▂▂▂▁▁▁▁▁▁▁▂▁▂▂▂▂▃▁▂▂▂▃▄▄▄▅▅▅▁█
train/grad_norm,▁▁▁▁▂▁▁▁▃▁▁▁▄▁▂▂▁▁▄▂▁▁▂▃▇▂▁▁▁▁▁█▁▁▂▁▁▁▂▁
train/learning_rate,█▁▁▁▁▁▁▁▁▁████▇████▇▂▃███████▇▆▆▆▅▅▃▃▂██
train/loss,▁▃▃▄▁▃▁▂▄▁▃▃▃▂█▄▂▅▃▆▂▆▄▃█▂▇▂▄▆▇▂▄▂▂▅▂▄▅▅
total_flos,0
train/epoch,1
train/global_step,500
train/grad_norm,1.6423
train/learning_rate,0
train/loss,1.8454


## Setup

In [ ]:
#installs
%%capture
%pip install -U transformers
%pip install -U datasets
%pip install -U accelerate
%pip install -U peft
%pip install -U trl
%pip install -U bitsandbytes
%pip install -U wandb

In [ ]:
#imports
from transformers import (
    AutoModelForCausalLM,
    AutoModelForQuestionAnswering,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    DefaultDataCollator,
    Trainer,
    pipeline,
    logging,
)
from peft import (
    LoraConfig,
    PeftModel,
    VeraConfig,
    PromptEncoder,
    PromptEncoderConfig,
    PrefixTuningConfig,
    LNTuningConfig,
    LoHaConfig,
    TaskType,
    prepare_model_for_kbit_training,
    get_peft_model,
)
import os, torch, wandb
from datasets import load_dataset
from trl import SFTTrainer, setup_chat_format

In [ ]:
#log in to hugging face and wandb
from google.colab import userdata
from huggingface_hub import notebook_login
huggingface_key = userdata.get('huggingface')
notebook_login(huggingface_key)

wandb_key = userdata.get('wandb_key')
wandb.login(key=wandb_key)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: alina775 (deep_learning_final_project). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
llama_base_model = "meta-llama/Llama-3.2-3B"

medical_chatbot_dataset_name = "ruslanmv/ai-medical-chatbot"
math_dataset_name = "nvidia/OpenMathInstruct-2"

quantisation_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True
)

In [ ]:
def run_training(base_model, config, dataset_name, dataset_size, data_row_name_user, data_row_name_assistant, output_directory, learning_rate, quantisation_config = None):
  #setup model
  if quantisation_config:
    model = AutoModelForCausalLM.from_pretrained(base_model, device_map = "auto", quantization_config = quantisation_config)
  else:
    model = AutoModelForCausalLM.from_pretrained(base_model, device_map = "auto") #change model
  peft_config = config #change peft method
  model = get_peft_model(model, peft_config)
  model.print_trainable_parameters()

  #load dataset
  dataset = load_dataset(dataset_name, split="train[:10000]").shuffle(seed=42).select(range(dataset_size)).flatten_indices()
  tokenizer = AutoTokenizer.from_pretrained(base_model)
  tokenizer.chat_template = None
  model, tokenizer = setup_chat_format(model, tokenizer)

  def format_chat_template(row):
      row_json = [{"role": "user", "content": row[data_row_name_user]},
                 {"role": "assistant", "content": row[data_row_name_assistant]}]
      row["text"] = tokenizer.apply_chat_template(row_json, tokenize=False)
      return row

  dataset = dataset.map(
      format_chat_template
  )
  dataset = dataset.train_test_split(test_size=0.1)

  training_arguments = TrainingArguments(
    output_dir = output_directory,
    learning_rate=learning_rate,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=3,
    weight_decay=0.01,
    eval_strategy="steps",
    save_strategy="steps",
    logging_steps=60,
    eval_steps=60,
    save_steps=120,
    report_to="wandb",
    load_best_model_at_end=True,
    gradient_accumulation_steps=8,
    warmup_steps=120,
    fp16=True
  )

  trainer = SFTTrainer(
    model=model,
    max_seq_length = 512,
    peft_config=peft_config,
    dataset_text_field="text",
    args=training_arguments,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    processing_class = tokenizer
  )

  return trainer.train()

## testing

### lora

- no quantisation
- llama 3 3b
- dataset: medical
- dataset size: 1000
- peft: lora


fails when it reaches the "model, tokenizer = setup_chat_format(model, tokenizer)" line. This is before the training occurs, meaning it is not possible to train without quantisation, eleminating vera as a peft method.

In [ ]:
run = wandb.init(
    project='Deep learning final project', # change
    job_type="training",
    anonymous="allow",
    name = "lora_medical_1000_llama"
)

peft_config = LoraConfig(
    r=8,
    inference_mode = False,
    lora_alpha=32,
    lora_dropout=0.1,
    task_type="CAUSAL_LM"
)
run_training(llama_base_model, peft_config, medical_chatbot_dataset_name, 1000, "Patient", "Doctor", "newModelLoRA", 1e-3)
wandb.finish()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

trainable params: 2,293,760 || all params: 3,215,043,584 || trainable%: 0.0713


The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.47 GiB. GPU 0 has a total capacity of 14.75 GiB of which 1.18 GiB is free. Process 36556 has 13.56 GiB memory in use. Of the allocated memory 13.45 GiB is allocated by PyTorch, and 569.50 KiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

- quantisation
- llama 3 3b
- dataset: medical
- dataset size: 1000
- peft: lora

In [ ]:
run = wandb.init(
    project='Deep learning final project', # change
    job_type="training",
    anonymous="allow",
    name = "lora_quantisation_medical_1000_llama"
)

peft_config = LoraConfig(
    r=8,
    inference_mode = False,
    lora_alpha=32,
    lora_dropout=0.1,
    task_type="CAUSAL_LM"
)
run_training(llama_base_model, peft_config, medical_chatbot_dataset_name, 1000, "Patient", "Doctor", "newModelLoRA", 1e-3, quantisation_config)
wandb.finish()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

trainable params: 2,293,760 || all params: 3,215,043,584 || trainable%: 0.0713


The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length, dataset_text_field. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:300: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:328: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


...

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss,Validation Loss
60,2.374000,1.895292
120,1.661800,1.615440
180,1.615000,1.565504
240,1.473000,1.560329
300,1.326800,1.558113


/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


eval/loss,█▂▁▁▁
eval/runtime,▁██▃▁
eval/samples_per_second,█▁▁▆█
eval/steps_per_second,█▁▁▆█
train/epoch,▁▁▃▃▄▄▆▆▇▇█
train/global_step,▁▁▃▃▄▄▆▆▇▇█
train/grad_norm,█▄▁▃▂
train/learning_rate,▄█▆▃▁
train/loss,█▃▃▂▁
eval/loss,1.55811
eval/runtime,31.9679


- quantisation
- llama 3 3b
- dataset: math
- dataset size: 1000
- peft: lora

In [ ]:
run = wandb.init(
    project='Deep learning final project', # change
    job_type="training",
    anonymous="allow",
    name = "lora_quantisation_math_1000_llama"
)

peft_config = LoraConfig(
    r=8,
    inference_mode = False,
    lora_alpha=32,
    lora_dropout=0.1,
    task_type="CAUSAL_LM"
)
run_training(llama_base_model, peft_config, math_dataset_name, 1000, "problem", "generated_solution", "newModelLoRA", 1e-3, quantisation_config)
wandb.finish()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

trainable params: 2,293,760 || all params: 3,215,043,584 || trainable%: 0.0713


Resolving data files:   0%|          | 0/32 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/32 [00:00<?, ?it/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/900 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Step,Training Loss,Validation Loss
60,0.896000,0.848901
120,0.751700,0.831281
180,0.708800,0.823207
240,0.691500,0.816346
300,0.575200,0.817383


/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


eval/loss,█▄▂▁▁
eval/runtime,█▂▁▇▁
eval/samples_per_second,▁▇█▂▇
eval/steps_per_second,▁▇█▂▇
train/epoch,▁▁▃▃▄▄▆▆▇▇█
train/global_step,▁▁▃▃▄▄▆▆▇▇█
train/grad_norm,█▄▇▁▂
train/learning_rate,▄█▆▃▁
train/loss,█▅▄▄▁
eval/loss,0.81738
eval/runtime,34.2184


### p-tuning

- quantisation
- llama 3 3b
- dataset: medical
- dataset size: 1000
- peft: p-tuning

In [ ]:
run = wandb.init(
    project='Deep learning final project', # change
    job_type="training",
    anonymous="allow",
    name = "p-tuning_quantisation_medical_1000_llama"
)

peft_config = PromptEncoderConfig(
    peft_type="P_TUNING",
    task_type="CAUSAL_LM",
    num_virtual_tokens=20,
    token_dim=3072,
    num_transformer_submodules=1,
    num_attention_heads=12,
    num_layers=12,
    encoder_reparameterization_type="MLP",
    encoder_hidden_size=3072,
)

run_training(llama_base_model, peft_config, medical_chatbot_dataset_name, 1000, "Patient", "Doctor", "newModelLoRA", 1e-3, quantisation_config)
wandb.finish()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

trainable params: 28,382,208 || all params: 3,241,132,032 || trainable%: 0.8757


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length, dataset_text_field. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:300: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:328: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Step,Training Loss,Validation Loss
60,2.691400,2.397743
120,2.378800,2.320283
180,2.248100,2.213115
240,2.017200,2.051843
300,1.783300,1.917007


/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


eval/loss,█▇▅▃▁
eval/runtime,▇▆▅▁█
eval/samples_per_second,▂▃▄█▁
eval/steps_per_second,▂▃▄█▁
train/epoch,▁▁▃▃▄▄▆▆▇▇█
train/global_step,▁▁▃▃▄▄▆▆▇▇█
train/grad_norm,█▂▂▇▁
train/learning_rate,▄█▆▃▁
train/loss,█▆▅▃▁
eval/loss,1.91701
eval/runtime,31.814


- quantisation
- llama 3 3b
- dataset: math
- dataset size: 1000
- peft: p-tuning

In [ ]:
run = wandb.init(
    project='Deep learning final project', # change
    job_type="training",
    anonymous="allow",
    name = "p-tuning_quantisation_math_1000_llama"
)

peft_config = PromptEncoderConfig(
    peft_type="P_TUNING",
    task_type="CAUSAL_LM",
    num_virtual_tokens=20,
    token_dim=3072,
    num_transformer_submodules=1,
    num_attention_heads=12,
    num_layers=12,
    encoder_reparameterization_type="MLP",
    encoder_hidden_size=3072,
)

run_training(llama_base_model, peft_config, math_dataset_name, 1000, "problem", "generated_solution", "newModelLoRA", 1e-3, quantisation_config)
wandb.finish()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

trainable params: 28,382,208 || all params: 3,241,132,032 || trainable%: 0.8757


Resolving data files:   0%|          | 0/32 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/32 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length, dataset_text_field. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:300: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:328: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


...

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss,Validation Loss
60,0.939600,0.808096
120,0.810300,0.783344
180,0.792900,0.767400
240,0.793500,0.757177
300,0.772800,0.750713
360,0.771200,0.747320
420,0.762400,0.741458
480,0.756600,0.740422
540,0.751600,0.737627


/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetunin

eval/loss,█▆▄▃▂▂▁▁▁
eval/runtime,▃██▆▁▃▂▂▇
eval/samples_per_second,▆▁▁▃█▅▇▇▂
eval/steps_per_second,▆▁▁▃█▅▇▇▂
train/epoch,▁▁▂▂▃▃▄▄▄▄▅▅▆▆▇▇███
train/global_step,▁▁▂▂▃▃▄▄▄▄▅▅▆▆▇▇███
train/grad_norm,█▄▁▁▂▂▁▁▁
train/learning_rate,▄█▇▆▅▄▃▂▁
train/loss,█▃▃▃▂▂▁▁▁
eval/loss,0.73763
eval/runtime,35.6783


### prefix-tuning

- quantisation
- llama 3 3b
- dataset: medical
- dataset size: 1000
- peft: prefix-tuning

In [ ]:
run = wandb.init(
    project='Deep learning final project', # change
    job_type="training",
    anonymous="allow",
    name = "prefix_quantisation_medical_1000_llama"
)

peft_config = PrefixTuningConfig(
    peft_type="PREFIX_TUNING",
    task_type="CAUSAL_LM",
    num_virtual_tokens=400,
    token_dim=3072,
    num_transformer_submodules=1,
    encoder_hidden_size=3072
)

run_training(llama_base_model, peft_config, medical_chatbot_dataset_name, 1000, "Patient", "Doctor", "newModelLoRA", 1e-3, quantisation_config)
wandb.finish()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

trainable params: 22,937,600 || all params: 3,235,687,424 || trainable%: 0.7089


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length, dataset_text_field. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:300: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:328: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Step,Training Loss,Validation Loss
60,8.920900,7.159659
120,6.404400,5.645657
180,5.055500,4.540845
240,4.133300,3.817180
300,3.608800,3.569819


/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


eval/loss,███▆▄▂▁▁
eval/runtime,▁▁▁▅█▄▄▄
eval/samples_per_second,███▄▁▅▄▅
eval/steps_per_second,███▄▁▅▄▅
train/epoch,▁▁▁▁▁▁▁▁▃▃▄▄▆▆▇▇█
train/global_step,▁▁▁▁▁▁▁▁▃▃▄▄▆▆▇▇█
train/grad_norm,▆█▄▃▂▃▂▁
train/learning_rate,▄▄▄▄█▆▃▁
train/loss,███▇▄▃▂▁
eval/loss,3.56982
eval/runtime,30.9388


### LayerNorm

- quantisation
- llama 3 3b
- dataset: medical
- dataset size: 1000
- peft: layernorm tuning

In [ ]:
run = wandb.init(
    project='Deep learning final project', # change
    job_type="training",
    anonymous="allow",
    name = "layerNorm_quantisation_medical_1000_llama"
)

peft_config = LNTuningConfig(
    task_type=TaskType.CAUSAL_LM,
)

run_training(llama_base_model, peft_config, medical_chatbot_dataset_name, 1000, "Patient", "Doctor", "newModelLoRA", 1e-3, quantisation_config)
wandb.finish()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

trainable params: 175,104 || all params: 3,212,924,928 || trainable%: 0.0054


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length, dataset_text_field. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:300: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:328: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Step,Training Loss,Validation Loss
60,2.622700,2.416605
120,2.288400,2.184773
180,2.041000,2.038435
240,1.826400,1.926585
300,1.705800,1.861850


/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


eval/loss,█▅▃▂▁
eval/runtime,▃█▅▁▄
eval/samples_per_second,▆▁▄█▅
eval/steps_per_second,▆▁▄█▅
train/epoch,▁▁▃▃▄▄▆▆▇▇█
train/global_step,▁▁▃▃▄▄▆▆▇▇█
train/grad_norm,█▁█▂▆
train/learning_rate,▄█▆▃▁
train/loss,█▅▄▂▁
eval/loss,1.86185
eval/runtime,31.2974


## Notes for me

- change model
- change dataset
- change dataset size
- change paramters?
- change task type?
- quantised vs not quantised?

peft techniques
- lora
- qlora
- vera
- p-turning
- prefix tuning
- layernorm tuning


Aim: make the best generalised fine tuning method for LLMs that works quickly on regular user devices, using the PEFT libary on huggingface


initial eliminations:
- llama3 3B can only be loaded when it is qunatised to avoid memory errors
- therefore vera eliminated
